# 線形代数

In [1]:
using LinearAlgebra

### ベクトル
Juliaは列ベクトルが基本．

$$
\mathbf{a}=[a_1, a_2, \ldots, a_n]^\top
$$

In [2]:
a = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

### 行列

$$
\begin{aligned}
\mathbf{A} =\left[\begin{array}{ccc}
a_{11} & \cdots & a_{1n} \\
\vdots & \ddots & \vdots  \\
a_{1n} & \cdots & a_{nn}
\end{array}\right]
\end{aligned} \in \mathbb{R}^n
$$

行列$\mathbf{A}$の$(i, j)$成分を$\mathbf{A}_{ij}=a_{ij}$とする．

In [3]:
A = []

Any[]

## 要素ごとの演算
### 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある．これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある．以下では行ごとの和を1にする．

In [4]:
W = rand(3,3)

3×3 Matrix{Float64}:
 0.926253  0.353171   0.343478
 0.62924   0.0944723  0.917437
 0.680284  0.170864   0.654275

In [5]:
Wnormed = W ./ sum(W, dims=1)

3×3 Matrix{Float64}:
 0.414287  0.571006  0.179344
 0.281441  0.152742  0.479032
 0.304272  0.276252  0.341624

In [6]:
println(sum(Wnormed, dims=1))

[1.0 1.0 1.0]


### 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる．まず，2つの行列A, Bを用意する．

In [7]:
A = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [8]:
B = [4 5 6; 7 8 9]

2×3 Matrix{Int64}:
 4  5  6
 7  8  9

### 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と，`[ ]`を使うやり方がある．

In [9]:
H1 = hcat(A,B)

2×5 Matrix{Int64}:
 1  2  4  5  6
 3  4  7  8  9

In [10]:
H2 = [A B]

2×5 Matrix{Int64}:
 1  2  4  5  6
 3  4  7  8  9

なお，MATLABのように次のようにすると正しく結合はされない．

In [11]:
H3 = [A, B]

2-element Vector{Matrix{Int64}}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 垂直結合 (Vertical concatenation)

In [12]:
V1 = vcat(A, B')

5×2 Matrix{Int64}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [13]:
V2 = [A; B']

5×2 Matrix{Int64}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [14]:
[V2 [A;B']]

5×4 Matrix{Int64}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9

## 配列に新しい軸を追加
要はnumpyでの`A[None, :]`や`A[np.newaxis, :]`のようなことがしたい場合．やや面倒だが，`reshape`を使うか，`[CartesianIndex()]`を用いる．

In [15]:
v = rand(3)

3-element Vector{Float64}:
 0.9225597515419179
 0.658120481093438
 0.41401801671066496

In [16]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Matrix{Float64}:
 0.92256  0.65812  0.414018

### 単位行列

In [17]:
I

UniformScaling{Bool}
true*I

In [18]:
I(3)

3×3 Diagonal{Bool, Vector{Bool}}:
 1  ⋅  ⋅
 ⋅  1  ⋅
 ⋅  ⋅  1

### 対角行列

### 線形行列方程式

$$
\mathbf{A}\mathbf{x}=\mathbf{b}
$$

$\mathbf{A}$が正則の場合，逆行列が存在し，

$$
\mathbf{x}=\mathbf{A}^{-1}\mathbf{b}
$$

In [33]:
A = rand(2,2)
b = rand(2)

2-element Vector{Float64}:
 0.5002132597166149
 0.03509562556923285

In [34]:
x = inv(A) * b

2-element Vector{Float64}:
 -0.1805788303672428
  0.8082531148175677

Juliaではバックスラッシュ演算子 `\`を用いることで明示的に逆行列を計算せずに解を求めることができる．

In [35]:
x = A \ b

2-element Vector{Float64}:
 -0.1805788303672428
  0.8082531148175676

## Roth's column lemma

Roth's column lemmaは，例えば，$A, B, C$が与えられていて，$X$を未知とするときの方程式 $AXB = C$を考えると，この方程式は

$$
(B^\top \otimes A)\text{vec}(X) = \text{vec}(AXB)=\text{vec}(C)
$$

の形に書き下すことができる，というものである．$\text{vec}(\cdot)$はvec作用素（行列を列ベクトル化する作用素）である．`vec(X) = vcat(X...)`で実現できる．Roth's column lemmaを用いれば，$AXB = C$の解は

$$
X = \text{vec}^{-1}\left((B^\top \otimes A)^{-1}\text{vec}(C)\right)
$$

として得られる．ただし，$\text{vec}(\cdot)^{-1}$は列ベクトルを行列に戻す作用素(inverse of the vectorization operator)である．`reshape()`で実現できる．2つの作用素をまとめると，

$$
\begin{align}
\text{vec} &: R^{m\times n}\to R^{mn}\\
\text{vec}^{−1} &: R^{mn}\to R^{m×n}
\end{align}
$$

であり，$\text{vec}^{−1}\left(\text{vec}(X)\right)=X\ (\text{for all}\ X\in R^{m\times n})，\text{vec}\left(\text{vec}^{−1}(x)\right)=x\ (\text{for all}\ x \in R^{mn})$となる．

In [21]:
using LinearAlgebra, Kronecker, Random

In [22]:
m = 4
A = randn(m, m)
B = randn(m, m)
C = convert(Array{Float64}, reshape(1:16, (m, m)))

4×4 Matrix{Float64}:
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

In [23]:
X = reshape((B' ⊗ A) \ vec(C), (m, m))

4×4 Matrix{Float64}:
  25.5193   -29.5701     5.09719  -18.3365
 -61.7388    71.768    -12.3934    44.6203
   5.53255   -6.36443    1.09258   -3.92298
   6.70725   -7.05331    1.14599   -4.0074

In [24]:
A * X * B

4×4 Matrix{Float64}:
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

### 配列の1次元化
配列を一次元化(flatten)する方法．まずは3次元配列を作成する．

In [25]:
B = rand(2, 2, 2)

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.915692  0.18149
 0.132838  0.0376235

[:, :, 2] =
 0.807789  0.297739
 0.569867  0.705417

用意されている`flatten`を素直に用いると次のようになる．

In [26]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Vector{Float64}:
 0.9156924090979948
 0.13283838898511735
 0.18149003405285813
 0.03762345094967079
 0.8077889408656388
 0.5698673388911334
 0.297739498855886
 0.7054169935372759

ただし，単に`B[:]`とするだけでもよい．

In [27]:
B[:]

8-element Vector{Float64}:
 0.9156924090979948
 0.13283838898511735
 0.18149003405285813
 0.03762345094967079
 0.8077889408656388
 0.5698673388911334
 0.297739498855886
 0.7054169935372759

### reshapeにおける残りの次元の指定
numpyにおいては(2, 3, 5)次元の配列に対し，reshape(-1, 5)を行うと(6, 5)次元の配列となった．これと同様なことは，Juliaでは:を使うことで実装できる．

In [28]:
a = rand(2,3,5)
b = reshape(a, (:, 5))

6×5 Matrix{Float64}:
 0.286014   0.923302  0.174386   0.10705    0.744143
 0.457232   0.234084  0.149783   0.684196   0.21295
 0.308209   0.712286  0.903116   0.487475   6.84179e-6
 0.0566201  0.526367  0.235321   0.0381212  0.336639
 0.0933628  0.980388  0.0258844  0.0371784  0.849971
 0.384149   0.628456  0.077691   0.970843   0.678124

## Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると，1次元配列のリストを2次元配列に変換できた．Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる．`...`はsplat operatorと呼ばれる．

In [29]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : Vector{Vector{Float64}}
Size : (5,)


In [30]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : Adjoint{Float64, Matrix{Float64}}
Size : (5, 3)


以下は多次元配列の場合．`cat(...)`で配列を結合し，`permitedims`で転置する．

In [31]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Vector{Array{Float64, 3}}
Size : (6,)


In [32]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64, 4}
Size : (6, 3, 4, 5)
